# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# conn = sqlite3.connect("Db-IMDB-Assignment.db")
conn = sqlite3.connect("/content/drive/My Drive/Colab Notebooks/SQL Assignment/Db-IMDB-Assignment.db")

## Sample Code

In [4]:
%%time
# Write your sql query below

query = """
        SELECT TRIM(Movie.title) AS 'Movie_Name'
        FROM Movie
        WHERE Movie.rating < 3
        
        """

q = pd.read_sql_query(query, conn)    
print(q.shape)
q.head()

(85, 1)
CPU times: user 66.9 ms, sys: 19.2 ms, total: 86.1 ms
Wall time: 156 ms


In [5]:
q.head()

,Movie_Name
0,Mastizaade
1,Dragonball Evolution
2,Loveyatri
3,Race 3
4,Gunday


##Columns identified to be Cleansed for Querying

SQLlite is case-sensitive

Ex:

select * from Person 
where gender ='Male'

select * from Person 
where gender = 'MALE'    

are different

1.   PID in M_Cast table mainly
2.   Year in Movie table
3.   Note: Gender has 3 values (Male, Female, **None**)



In [6]:
%%time
# Write your sql query below

query = """
        SELECT TRIM(Movie.title) AS 'Movie_Name'
        FROM Movie
        WHERE Movie.rating < 3
        
        """

q = pd.read_sql_query(query, conn)    
print(q.shape)
q.head()

(85, 1)
CPU times: user 1.98 ms, sys: 781 µs, total: 2.76 ms
Wall time: 2.84 ms


In [7]:
q.head()

,Movie_Name
0,Mastizaade
1,Dragonball Evolution
2,Loveyatri
3,Race 3
4,Gunday


## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [8]:
%%time
# Write your sql query below
#https://www.w3resource.com/sqlite/core-functions-substr.php


query = """
          SELECT p.name AS director, 
                m.title, 
                cast(Substr(Trim(m.year), -4) as int) as year 
          FROM   movie m 
                INNER JOIN m_director md 
                        ON m.mid = md.mid 
                INNER JOIN Person p 
                        ON md.pid = p.pid 
                INNER JOIN m_genre mg 
                        ON m.mid = mg.mid 
                INNER JOIN genre g 
                        ON mg.gid = g.gid 
          WHERE  cast(Substr(Trim(m.year), -4) as int)%4 = 0 
                AND g.name LIKE '%Comedy%'
          """ 
        
q1 = pd.read_sql_query(query, conn)    
print(q1.shape)

(232, 3)
CPU times: user 57.9 ms, sys: 5.37 ms, total: 63.3 ms
Wall time: 68 ms


In [9]:
q1.head()

,director,title,year
0,Milap Zaveri,Mastizaade,2016
1,Danny Leiner,Harold & Kumar Go to White Castle,2004
2,Anurag Kashyap,Gangs of Wasseypur,2012
3,Frank Coraci,Around the World in 80 Days,2004
4,Griffin Dunne,The Accidental Husband,2008


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [10]:
%%time
# Write your sql query below

query = """
        SELECT p.NAME 
        FROM   movie m 
              INNER JOIN m_cast mc 
                      ON m.mid = mc.mid 
              INNER JOIN Person p 
                      ON Trim(mc.pid) = p.pid
        WHERE  title = 'Anand' 
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)

(17, 1)
CPU times: user 158 ms, sys: 2.83 ms, total: 161 ms
Wall time: 168 ms


In [11]:
q2.head()

,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Brahm Bhardwaj
3,Ramesh Deo
4,Seema Deo


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [12]:
%%time
# Write your sql query below

query = """
        SELECT Name from Person p
        WHERE trim(pid) in   
        (SELECT distinct trim(PID) from Movie m
        INNER JOIN M_Cast mc
        ON m.mid = mc.mid
        WHERE cast(Substr(Trim(m.year), -4) as int)  < 1970
        INTERSECT
        SELECT distinct trim(PID) from Movie m
        INNER JOIN M_Cast mc
        ON m.mid = mc.mid
        WHERE cast(Substr(Trim(m.year), -4) as int)  > 1990 ) 
        """

q3 = pd.read_sql_query(query, conn)    
print(q3.shape)

(300, 1)
CPU times: user 266 ms, sys: 5.42 ms, total: 271 ms
Wall time: 272 ms


In [13]:
q3.head()

,Name
0,Rishi Kapoor
1,Amitabh Bachchan
2,Asrani
3,Zohra Sehgal
4,Parikshat Sahni


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [14]:

%%time
# Write your sql query below

query = """
        SELECT Name as Director, Count(Name) as Movies_directed from 
        (SELECT m.mid, md.pid, p.Name from Movie m
        INNER JOIN M_Director md
        ON m.mid = md.mid
        INNER JOIN Person p
        ON md.pid = p.pid)  A
        GROUP BY Director
        ORDER BY Movies_directed DESC
        
        """

q4 = pd.read_sql_query(query, conn)    
print(q4.shape)

(1459, 2)
CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 62.2 ms


In [15]:
q4.head()

,Director,Movies_directed
0,David Dhawan,39
1,Mahesh Bhatt,36
2,Ram Gopal Varma,30
3,Priyadarshan,30
4,Vikram Bhatt,29


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [16]:
# Step1: Find the Movies list which male actors had their role
# Step2: Remove this list from entire list of Movies then you will get movies which has only female actors (OR) no male actors in it
# Step 3: use this MID's to populate the reuired data

In [17]:

%%time
# Write your sql query below


query = """
          SELECT substr(Year,-4) as Year,count(*) as Movies from Person p
          INNER JOIN M_cast mc
          ON p.pid = trim(mc.pid)
          INNER JOIN Movie m 
          ON mc.mid = m.mid
          WHERE m.mid not in (
            SELECT distinct m.mid from Movie m
            INNER JOIN M_Cast mc
            ON m.mid = mc.mid
            INNER JOIN Person p
            ON trim(mc.pid) = p.pid
            WHERE gender <> 'Female' )

          and gender = 'Female'
          GROUP BY cast(Substr(Trim(m.year), -4) as int) 


        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)

(4, 2)
CPU times: user 455 ms, sys: 6.07 ms, total: 461 ms
Wall time: 471 ms


In [18]:
q5a.head()

,Year,Movies
0,1939,1
1,1999,11
2,2000,10
3,2018,2


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [19]:

%%time
# Write your sql query below
# https://stackoverflow.com/questiONs/8305613/cONverting-int-to-real-in-sqlite
# https://stackoverflow.com/questiONs/41427207/using-with-clause-in-sqlite3-are-the-results-of-with-clause-saved-and-reused

# A is Ans for 5A query
# B total movies per year

query = """
          WITH 

          A as (SELECT cast(substr(Year,-4) as int) as Year,count(*) as Female_Movies from Person p
                INNER JOIN M_cast mc
                ON p.pid = trim(mc.pid)
                INNER JOIN Movie m 
                ON mc.mid = m.mid
                WHERE m.mid not in (
                  SELECT distinct m.mid from Movie m
                  INNER JOIN M_Cast mc
                  ON m.mid = mc.mid
                  INNER JOIN Person p
                  ON trim(mc.pid) = p.pid
                  WHERE gender <> 'Female' ) 

                and gender = 'Female'
                GROUP BY cast(substr(Year,-4) as int) ) ,

          B as (SELECT cast(substr(Year,-4) as int) as Year,count(*) as Movies from Person p
                  INNER JOIN M_cast mc
                  ON p.pid = trim(mc.pid)
                  INNER JOIN Movie m 
                  ON mc.mid = m.mid
                  GROUP BY cast(substr(Year,-4) as int))


          SELECT A.Year as Year, A.Female_Movies ,B.Movies,
          (A.Female_Movies*100.00/B.Movies) as 'Fem_Movies%'
          from A INNER JOIN B
          ON A.year = B.year


        """

q5b = pd.read_sql_query(query, conn)    
print(q5b.shape)

(4, 4)
CPU times: user 609 ms, sys: 15.2 ms, total: 624 ms
Wall time: 633 ms


In [20]:
q5b.head()

,Year,Female_Movies,Movies,Fem_Movies%
0,1939,1,45,2.222222
1,1999,11,1628,0.675676
2,2000,10,1480,0.675676
3,2018,2,2942,0.067981


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [21]:

%%time
# Write your sql query below
# Movie_cast table will cONsists of unique combinatiON of MovieID,ActorID
#like   tt123,Actor123
# Note: Movie Names may be duplicate in Movie table

query = """
        WITH 

        Movie_Cast as
          (SELECT m.mid as mid ,trim(mc.pid) as pid  from Movie m
            INNER JOIN M_cast mc
            ON m.mid = mc.mid
            GROUP BY m.mid,trim(mc.pid) ) ,
        A as (SELECT Movie_Cast.mid, count(*) as Cast_cnt from Movie_Cast 
            GROUP BY mid)
        
        SELECT m.title, Cast_cnt from A
        INNER JOIN 
            Movie m 
        ON A.mid = m.mid
        WHERE Cast_cnt = ( SELECT max(Cast_cnt) from A ) 
        """

q6 = pd.read_sql_query(query, conn)    
print(q6.shape)

(1, 2)
CPU times: user 457 ms, sys: 40.1 ms, total: 497 ms
Wall time: 509 ms


In [22]:
q6.head()

,title,Cast_cnt
0,Ocean's Eight,238


## Q7 --- A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [23]:
%%time
# Write your sql query below
# https://stackoverflow.com/questions/32953639/how-to-type-cast-in-sqlite3
# https://www.w3resource.com/sqlite/core-functions-substr.php
# https://www.techonthenet.com/sqlite/between.php
# First get the decades table - Year column has to cleansed
# group by cast(substr(Year,-4) as int))  this is important else you will get the wrng counts

query = """
        WITH
        Year_decade as (
          SELECT DISTINCT cast(Substr(Trim(m.year), -4) as int) as Year,
          (cast(Substr(Trim(m.year), -4) as int)+0) ||'-'|| (cast(Substr(Trim(m.year), -4) as int)+9) as decade,
          cast(Substr(Trim(m.year), -4) as int)+0 as decade_start,
          cast(Substr(Trim(m.year), -4) as int)+9 as decade_end
          from Movie m ),

        Year_Movie_cnt as
        (SELECT cast(Substr(Trim(m.year), -4) as int) as year, count(*) as movie_cnt
        from Movie m 
        GROUP BY cast(substr(Year,-4) as int)),

        Decade_movieCnt as
        (SELECT yd.decade,sum(movie_cnt) as total_cnt from Year_decade yd
        INNER JOIN Year_Movie_cnt ymc
        ON ymc.year >= yd.decade_start
        and ymc.year <= yd.decade_end
        GROUP BY decade
        order by yd.decade,ymc.year)

        SELECT * from Decade_movieCnt
        WHERE total_cnt = (SELECT max(total_cnt) from Decade_movieCnt)

        """

q7 = pd.read_sql_query(query, conn)    
print(q7.shape)


(1, 2)
CPU times: user 19.3 ms, sys: 852 µs, total: 20.2 ms
Wall time: 21.5 ms


In [24]:
q7

,decade,total_cnt
0,2008-2017,1205


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [25]:
%%time
# Write your sql query below
# Trim Yash Chopra as it has sapces in db

query = """
        WITH
        Actor_movie_dir as
        (SELECT TRIM(mc.pid) as actor_id,m.mid,m.title,md.pid as dir_id  from M_cast mc
        INNER JOIN Movie m
        ON mc.mid = m.mid
        INNER JOIN M_Director md
        ON m.mid = md.mid
        INNER JOIN Person P
        ON trim(mc.pid) = p.pid),

        Actor_dir_MovCount as
        (SELECT actor_id,dir_id,count(*) cnt from Actor_movie_dir
        group by actor_id,dir_id
        order by cnt desc),

        Actor_maxMovies as
        (SELECT actor_id,max(cnt) as max_cnt from Actor_dir_MovCount
        group by actor_id)

        SELECT adm.actor_id,
        p.name as Actor_name,adm.dir_id,
        (SELECT Name from Person WHERE pid = adm.dir_id) as Director_name,
        adm.cnt 
        from Actor_dir_MovCount adm
        INNER JOIN Actor_maxMovies am
        ON adm.actor_id = am.actor_id
        and adm.cnt = am.max_cnt
        INNER JOIN Person p
        ON am.actor_id = p.pid
        
        WHERE adm.dir_id = (SELECT p.pid from Person p
                            WHERE trim(p.name) ='Yash Chopra')

        """

q8 = pd.read_sql_query(query, conn)    
print(q8.shape)

(245, 5)
CPU times: user 896 ms, sys: 21.2 ms, total: 917 ms
Wall time: 921 ms


In [26]:
q8.head()

,actor_id,Actor_name,dir_id,Director_name,cnt
0,nm0707271,Jagdish Raj,nm0007181,Yash Chopra,11
1,nm0471443,Manmohan Krishna,nm0007181,Yash Chopra,10
2,nm0407002,Iftekhar,nm0007181,Yash Chopra,9
3,nm0004434,Shashi Kapoor,nm0007181,Yash Chopra,7
4,nm0347901,Rakhee Gulzar,nm0007181,Yash Chopra,5


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [29]:

%%time
# Write your sql query below

query = """
        WITH 
        SRK_1 as 
        (SELECT  DISTINCT trim(mc.pid) as pid from M_Cast mc
        INNER JOIN Person p
        ON p.pid = trim(mc.pid)
        WHERE mid in (
                SELECT distinct mc.mid from M_cast mc
                INNER JOIN Person p
                ON trim(mc.pid) = p.pid
                WHERE trim(p.name) = 'Shah Rukh Khan')            
        and trim(p.name) != 'Shah Rukh Khan')

        SELECT  DISTINCT trim(mc.pid) as pid,p.name as name from M_Cast mc
        INNER JOIN Person p
        ON p.pid = trim(mc.pid)
        WHERE mc.mid in (
                      SELECT mid from M_cast mc
                      WHERE trim(mc.pid) in (
                                            SELECT pid from SRK_1 ))
        and trim(p.name) != 'Shah Rukh Khan'
        and trim(mc.pid) not in (select PID from SRK_1)

        
        """

q9 = pd.read_sql_query(query, conn)    
print(q9.shape)

(25698, 2)
CPU times: user 749 ms, sys: 11.5 ms, total: 760 ms
Wall time: 763 ms


In [30]:
q9.head()

,pid,name
0,nm2539953,Alicia Vikander
1,nm0922035,Dominic West
2,nm0324658,Walton Goggins
3,nm0943079,Daniel Wu
4,nm0000218,Kristin Scott Thomas
